## In case you need a reminder of why the local community is how it is

In [7]:
import numpy as np
import os

workdir = "/Volumes/WorkDrive/gimmeSAD"
os.chdir(workdir)

import Community



In [8]:
data = Community.Community()
data.set_metacommunity("metacommunity_LS4.txt")
data.prepopulate(mode="landbridge")

('total_inds', 1000300302)
('metacommunity', array([(0, 517471, 0.0005173156490759512), (1, 66, 6.598018601817837e-08),
       (2, 1, 9.996997881542177e-10), ...,
       (1634, 21854754, 0.021848192943962542),
       (1635, 138741, 0.0001386993483083043),
       (1636, 3255510, 0.0032545326573339374)], 
      dtype=[('uuid', '<i8'), ('abund', '<i8'), ('col_prob', '<f8')]))
[('uuid', '<i8'), ('abund', '<i8'), ('col_prob', '<f8')]
('init_community', array([[  2,   0,   0, ..., 247,   0,  24]]))
Initializing local community:
N species = 477
N individuals = 10000
('local_community', array([(0, 2, 1, True), (1, 0, 0, False), (2, 0, 0, False), ...,
       (1634, 247, 1, True), (1635, 0, 0, False), (1636, 24, 1, True)], 
      dtype=[('uuid', '<i8'), ('abund', '<i8'), ('col_time', '<i8'), ('is_founder', '?')]))


In [37]:
import random
%timeit random.choice(data.metacommunity["uuid"])
%timeit np.random.choice(data.metacommunity["uuid"])

10000 loops, best of 3: 22.6 µs per loop


In [263]:
## 10000 loops, best of 3: 165 µs per loop
def rand1(data):
    idx = (np.cumsum(data.local_community[np.nonzero(data.local_community["abund"])]["abund"])/float(data.K)).searchsorted(np.random.sample(1))[0]
    local_abund = np.nonzero(data.local_community["abund"])[0]
    #print(idx)#, len(local_abund), local_abund)
    idx = np.nonzero(data.local_community["abund"])[0][idx]
    #print(idx)
    return idx

## 10000 loops, best of 3: 91.6 µs per loop
def rand2(data):
    victim = np.random.multinomial(1, data.local_community["abund"]/float(data.K), size=1)
    idx = data.local_community['uuid'][victim[0].astype("bool")]
    return idx

import itertools, random

## 1000 loops, best of 3: 1.94 ms per loop
def rand3(data):
    local_abunds = [[x[0]] * x[1] for x in data.local_community[np.nonzero(data.local_community["abund"])]]
    ## flatten list
    local_abunds = list(itertools.chain.from_iterable(local_abunds))
    idx = random.choice(local_abunds)

## 10000 loops, best of 3: 118 µs per loop
def rand4(data):
    subset = data.local_community[np.nonzero(data.local_community["abund"])]
    lt = subset["abund"].cumsum()
    get = lt.searchsorted(random.randint(0, lt.max()))
    return get

## 10000 loops, best of 3: 105 µs per loop
## 10000 loops, best of 3: 33.8 µs per loop
def rand5(data):
    #idx = data.local_community[np.nonzero(data.local_community["abund"])]["abund"].cumsum().searchsorted(random.randint(0, data.K))
    idx = np.nonzero(data.local_community["abund"])[0].cumsum().searchsorted(random.randint(0, data.K))
    return idx